<h3>Install Library Open CV dan imutils</h3>

In [ ]:
!pip install opencv-python
!pip install imutils

<h3>Import library</h3>

In [ ]:
# import library keras
import keras

# import library open computer vision
import cv2

# import library komputasi numerik
import numpy as np

# import ilbrary pendukung open cv untuk keperluan manajemen data
from imutils import paths

import random

<h3>Load gambarnya</h3>

In [ ]:
# siapkan variabel kosong untuk menampung gambarnya
data = []

# siapkan variabel kosong untuk menampung label yang akan diprediksi
labels = []


# load dataset gambarnya dari path 'dataset/animals/'
dataset_path = 'datasets/animals/'
imagePaths = sorted(list(paths.list_images(dataset_path)))

# gambarnya dibuat random (shuffle)
random.seed(2)
random.shuffle(imagePaths)

# masukan data gambar hasil load tadi satu persatu ke dalam variabel data yang kosong tadi
for imagePath in imagePaths:
    
    # baca gambarnya
    img = cv2.imread(imagePath)
    
    # ubah gambarnya ke dalam grayscale (hitam putih) agar memiliki 1 channel (1-dimensional)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # ubah ukuran / dimensi gambarnya menjadi 32x32 px, serta convert ke dalam format vektor
    img_flat = cv2.resize(img_gray, (32,32)).flatten()
    
    # simpan / tumpuk gambar yang sudah dibaca tadi
    data.append(img_flat)    
    
    # baca labelnya dengan melihat path foldernya
    label = imagePath.split(os.path.sep)[-2]
    
    # simpan / tumpuk label yang sudah dibaca tadi 
    labels.append(label)

# ubah data label tadi menjadi format array agar lebih memudahkan dalam komputasi
lbl = np.array(labels)

# ubah var data menjadi array numpy agar lebih memudahkan dalam komputasi
dt = np.array(data, dtype='float32')

print('done')

<h3>Lihat salah satu gambarnya</h3>

In [ ]:
# import library matplotlib untuk melihat gambarnya
import matplotlib.pyplot as plt

# coba kita lihat sebuah gambar pada indeks ke-2 pada iamgePaths
gambar = imagePaths[2]

# baca gambarnya
gbr = cv2.imread(gambar)
gbr = cv2.resize(gbr, (100,100))

# tambahkan judul yang dilengkapi dengan nama file gambarnya + dimensi asli gambarnya 
plt.title('gambar = '+str(gambar.split('/')[3])+' ; ukuran = '+str(gbr.shape))

# munculkan 
plt.imshow(gbr)

<h3>Cek ukuran / dimensi gambarnya berapa</h3>

In [ ]:
print('dimensi        : ', dt.shape)
print('jumlah gambar  : ', dt.shape[0])
print('Ukuran gambar  : ', dt.shape[1])

3072 didapatkan dari 32 x 32 x 3

<h4>pecah data yang sudah diload tadi menjadi 2 bagian yaitu</h4>
<h4>1) bagian train</h4>
<li>X Train</li>
<li>Y Train</li>
<h4>2) bagian Test</h4>
<li>X Test</li>
<li>Y Test</li>
<h5>X = data input ; Y = label/output</h5>

In [ ]:
# import library untuk memecah datasetnya menjadi bagian train dan test
from sklearn.model_selection import train_test_split

# pecah datasetnya dengan jumlah data testing sebanyak 25% dari keseluruhan total data
x_train, x_test, y_train, y_test = train_test_split(dt, lbl, 
                                                    test_size=0.25, 
                                                    random_state=42)

# cek apakah sudah benar atau belum pemecaannya
print('x_train : ',x_train.shape)
print('x_test  : ', x_test.shape)
print('y_train : ', y_train.shape)
print('y_test  : ', y_test.shape)

<h3>Lakukan one hot enconding</h3>

In [ ]:
# import library untuk melakukan one-hot-encoding dengan memanfaatkan method LabelBinarizer
from sklearn.preprocessing import LabelBinarizer

# membuat object construktur
lb = LabelBinarizer()

# memulai melakukan one-hot-encoding untuk y_train dan y_test
y_train_encoded = lb.fit_transform(y_train)
y_test_encoded = lb.transform(y_test)

# cek nilainya
lb.classes_

### Normalisai Data

In [ ]:
# ubah tipe data gambarnya ke float
x_train = x_train.astype('float')
x_test = x_test.astype('float')

# normalisasi data gambarnya agar setiap nilai pixel memiliki rentang dari 0 sampai 1
x_train = x_train / 255
x_test = x_test / 255

<h3>Buat layer NN sederhana saja</h3>
<img src="https://i.ibb.co/0GyxYhB/nn-9.png" alt="nn-9" border="0" width="600px">

In [ ]:
# import layer model sebagai cara memembuat model NN-nya
from keras.models import Model

# import layer Input dan Linear layer (Dense layer)
from keras.layers import Dense, Input, Activation

# tentukan jumlah node dari input layer dan output layer
data_input  = 32 * 32 * 1
data_output = 3

# mulai membuat arsitektur NN dengan tipe fully connected layer
input_layer    = Input(shape=(data_input, ))
hidden_layer_1 = Dense(1024, activation='relu', name='hidden_layer_1')(input_layer)
hidden_layer_2 = Dense(512, activation='relu', name='hidden_layer_2')(hidden_layer_1)
output_layer   = Dense(data_output, activation='softmax', name='ouput_layer')(hidden_layer_2)
model          = Model(inputs=input_layer, outputs=output_layer)

# lihat ringkasan dari model yang suda kita buat
model.summary()

<h3> Atur jenis loss function, optimizer dan metrics </h3>

In [ ]:
# import library optimizer, sebagai contoh SGD optimizer
from keras.optimizers import SGD

# tentukan nilai learning-rate untuk optimizer yang digunakan
opt = SGD(lr=0.001)

# compile model dan tentukan fungsi loss, optimizer dan metriks pengujian model yang akan dilatih
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

<h3> Mulai training dengan 20 iterasi </h3>

In [ ]:
# memulai pelatihan
history = model.fit(x_train, y_train_encoded, 
                     validation_data=(x_test, y_test_encoded),
                     batch_size=32, 
                     epochs=20)

### Lihat hasil ujiannya (evaluation)

In [ ]:
# import metode classification report untuk melihat kinerja model yang dihasilkan
from sklearn.metrics import classification_report

# ambil nilai akurasinya
akurasi = model.evaluate(x_test, y_test_encoded)[0]

# ambil nilai loss-nya
loss = model.evaluate(x_test, y_test_encoded)[1]

# tampilkan akurasi dan lossnya
print()
print('akurasi : %.2f%%' % (akurasi))
print('loss    : %.2f%%' % loss, '\n')

# tampilkan laporan kinerja modelnya
prediksi = model.predict(x_test)
print(classification_report(y_test_encoded.argmax(axis = 1), prediksi.argmax(axis = 1), target_names=lb.classes_))

<h3> Lihat grafik akurasi dan loss nya </h3>

In [ ]:
# nilai-nilai loss-nya
loss = history.history['loss']

# nilai-nilai val_loss-nya
val_loss = history.history['val_loss']

# nilai-nilai akurasi-nya
acc = history.history['acc']

# nilai-nilai val_acc-nya
val_acc = history.history['val_acc']

# menentukan sumbu x nya dari 0 sampai dengan banyaknya jumlah nilai loss/acc
sumbu_x = np.arange(0, len(loss))

# memasukan library plotting (matplotlib)
import matplotlib.pyplot as plt

# grafik plot untuk melihat loss
plt.figure(figsize=(15,3))
plt.subplot(1,2,1)
plt.plot(sumbu_x, loss, label='loss')
plt.plot(sumbu_x, val_loss, label='val_loss')
plt.legend()

# grafik plot untuk melihat akurasinya
plt.subplot(1,2,2)
plt.plot(sumbu_x, acc, label='acc')
plt.plot(sumbu_x, val_acc, label='val_acc')
plt.legend()
plt.show()

### Simpan model dan label

In [ ]:
# simpan model keras / nn - nya ke dalam format h5
model.save('model_fc.h5')

# simpan label - nya ke dalam format pickle
import pickle
f = open('lable.pickle', 'wb')
f.write(pickle.dumps(lb))
f.close()
print('done')

In [ ]:
# import library pandas untuk manipulasi struktur datanya
import pandas as pd

# siapkan list comprehension untuk menampung nilai-nilai actual (sebenarnya) dari y_test
arr_y_test = [lb.classes_[y_test_encoded.argmax(1)[i]] for i in range(y_test.shape[0])]

# masukan list comprehension yang sudah dibuat ke dalam dataframe dengan pandas
y_test_df = pd.DataFrame(np.array(arr_y_test), columns=['y_test'])

# siapkan list comprehension untuk menampung nilai-nilai hasil prediksi
arr_pred_df = [lb.classes_[i] for i in model.predict(x_test).argmax(1)]

# masukan list comprehension yang sudah dibuat ke dalam dataframe 
pred_df = pd.DataFrame(np.array(arr_pred_df), columns=['prediksi'])

# gabungkan antara data nilai sebenarnya dengan hasil prediksi
perbandingan = y_test_df.join(pred_df, how='left')

# lihat perbandingannya
perbandingan.head()